<center><h1>kabra_shreyansh_FinalProject</h1></center>

Name: Shreyansh Kabra
<br>
Github Username: kabrashrey
<br>
USC ID: 5690865072

## 1. Transfer Learning for Image Classification

### (a) In this problem, we are trying to build a classifier that distinguishes images of nine types of waste.

In [3]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import cv2
import math
import shutil
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

# Check for GPU availability
tf.debugging.set_log_device_placement(True)
print("TensorFlow version:", tf.__version__)
print("GPU devices found:", tf.config.list_physical_devices('GPU'))




TensorFlow version: 2.16.2
GPU devices found: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# Constants and configurations
DATA_DIR = "../data/RealWaste"
SPLIT_DIR = "../data/RealWasteSplit"
TRAIN_SPLIT = 0.8
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

### (b) Data Exploration and Pre-processing

#### (i) & (ii) Resizing, padding, train-test-split, and one-hot encoding

In [5]:
# Resize and pad images to a fixed size
def pad_and_resize(image_path, target_size=IMG_SIZE):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image at {image_path} could not be read.")
    
    h, w = img.shape[:2]
    scale = min(target_size[1] / w, target_size[0] / h)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    padded = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)
    x_offset = (target_size[1] - new_w) // 2
    y_offset = (target_size[0] - new_h) // 2
    padded[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized

    return padded

# Split the dataset into training and testing sets
if os.path.exists(SPLIT_DIR):
    shutil.rmtree(SPLIT_DIR)

for class_name in sorted(os.listdir(DATA_DIR)):
    class_dir = os.path.join(DATA_DIR, class_name)
    if not os.path.isdir(class_dir):
        continue

    all_images = sorted(os.listdir(class_dir))
    train_size = int(len(all_images) * TRAIN_SPLIT)

    train_imgs = all_images[:train_size]
    test_imgs = all_images[train_size:]

    for split, split_imgs in zip(['train', 'test'], [train_imgs, test_imgs]):
        split_dir = os.path.join(SPLIT_DIR, split, class_name)
        os.makedirs(split_dir, exist_ok=True)
        
        for img in split_imgs:
            src = os.path.join(class_dir, img)
            dst = os.path.join(split_dir, img)
            try:
                output_img = pad_and_resize(src, IMG_SIZE)
                cv2.imwrite(dst, output_img)
            except Exception as e:
                print(f"Skipping {img}: {e}")
            shutil.copy(src, dst)

print("Padded and resized images saved to:", SPLIT_DIR)

Padded and resized images saved to: ../data/RealWasteSplit


In [6]:
# One-Hot Encoding
raw_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(SPLIT_DIR, 'train'),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    label_mode='int'
    )

raw_test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(SPLIT_DIR, 'test'),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    label_mode='int'
    )

class_names = raw_train_ds.class_names
NUM_CLASSES = len(class_names)

def one_hot_encode(image, label):
    """Convert integer labels to one-hot encoded format."""
    return image, tf.one_hot(label, depth=NUM_CLASSES)

train_ds = raw_train_ds.map(one_hot_encode).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = raw_test_ds.map(one_hot_encode).prefetch(buffer_size=tf.data.AUTOTUNE)

Found 3798 files belonging to 9 classes.
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
input: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
_EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
components_0: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
TensorSliceDataset: (TensorSliceDataset): /job:localhost/replica:0/task:0/device:CPU:0
handle_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
input: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
_EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
components_0: (_DeviceArg): /job:localhost/replica:0/task:0/device:CPU:0
Tensor

In [9]:
# Difference between raw and one-hot encoded labels
# for images, labels in raw_train_ds.take(1):
#     print("Raw labels (integer class indices):", labels.numpy())

# for images, labels in train_ds.take(1):
#     print("One-hot encoded labels:")
#     print(labels.numpy())

"""
Raw labels (integer class indices): 
[6 3 3 8 6 7 7 7 4 7 2 6 4 1 4 7 2 6 0 5 6 7 7 6 3 2 3 4 5 3 3 5]

One-hot encoded labels:
[[0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]]
 
"""

'\nRaw labels (integer class indices): \n[6 3 3 8 6 7 7 7 4 7 2 6 4 1 4 7 2 6 0 5 6 7 7 6 3 2 3 4 5 3 3 5]\n\nOne-hot encoded labels:\n[[0. 1. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 0. 0. 1.]\n [0. 0. 0. 0. 1. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 1. 0. 0. 0.]\n [0. 0. 0. 0. 0. 1. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 1. 0. 0.]\n [0. 0. 0. 0. 0. 0. 1. 0. 0.]\n [0. 0. 0. 0. 0. 0. 0. 1. 0.]\n [0. 0. 0. 0. 0. 0. 1. 0. 0.]\n [0. 0. 1. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1. 0. 0. 0. 0. 0.]\n [0. 1. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 0. 0. 1.]\n [0. 0. 0. 0. 0. 0. 0. 0. 1.]\n [0. 0. 1. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1. 0. 0. 0. 0. 0.]\n [0. 1. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 1. 0. 0.]\n [0. 0. 0. 0. 0. 0. 0. 0. 1.]\n [0. 0. 0. 1. 0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 1. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 1. 0. 0.]\n [1. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 1. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 0. 0. 1.]\n [0. 0. 0. 0. 1. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0. 0. 0. 1. 0.

### (b) Transfer Learning

#### (i)

##### (ii)

##### (iii)

##### (iv)

##### (v)

https://builtin.com/data-science/transfer-learning

